## Pyradiomics implemented in Google Colab

In [ ]:
!pip install radiomics

!pip install pyradiomics

In [ ]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib as plt
import SimpleITK as sitk
import numpy as np

j=1
dtadir= os.path.join(os.getcwd())
print(dtadir)

In [ ]:

feature_rows = []
for i in range(1,539):
  d_img=sitk.ReadImage(os.path.join("/content/drive/MyDrive/test_data/diastole"+str(i)+".png"), sitk.sitkInt8)
  im_arr = sitk.GetArrayFromImage(d_img)
  im_arr = np.expand_dims(im_arr, axis=0)
  im3d = sitk.GetImageFromArray(im_arr)
  d_gt=sitk.ReadImage(os.path.join('/content/drive/MyDrive/test_data_gt/gt'+str(i)+".png"), sitk.sitkInt8)
  print(np.unique(d_gt))

  im_arr = sitk.GetArrayFromImage(d_gt)
  im_arr = np.expand_dims(im_arr, axis=0)
  im3d_gt = sitk.GetImageFromArray(im_arr)
  params_path= os.path.join(dtadir, "Params.yaml")
  settings={'force2Ddimension': 1}
  settings['shape']=False
  settings['force2D']=1
  settings['shape2D']=1
  settings['binWidth'] = 20
  settings['sigma'] = [1, 2,3]


  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call
  if len(np.unique(d_gt))>1 and -35 in np.unique(d_gt):   # According to the pixel values vary value i.e -35, -127, -170, -255
    settings = {'label':4294967261}
    extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(additionalInfo=True, **settings)
    result = extractor.execute(im3d, im3d_gt)
    #print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
    #print('')
    print(j)
    j+=1
    print(np.unique(d_gt))

    row = {'filename': str(i)}

    for key, value in six.iteritems(result):
        row['LV'+ key] = value
    feature_rows.append(row)
    import pandas as pd
    # create a pandas DataFrame from the list of feature rows
    df = pd.DataFrame(feature_rows)


# save the DataFrame to a CSV file
df.to_csv('values_LV.csv', index=False)
